In [ ]:
from turtle import st
import dxfgrabber
import numpy
import csv

#DXFデータをcsvに出力するもの。
name = "kukei"
dxf = dxfgrabber.readfile(name+".dxf")
cirs = [e for e in dxf.entities if e.dxftype == 'CIRCLE'] #基準点１点（座標が１種だけ）とする。レイヤーは"0_KIJUN"だけ。
offsetxy = 0
for i in range(len(cirs)):
    ci = vars(cirs[i])
    #読み込んだ数値とUCSが対応していないので、規準となる円をdxfに配置して、基準からの座標に変換する。
    if ci['layer'] == "0_KIJUN": #0_KIJUNレイヤに円のオブジェクトを配置してそれを読む（１点のみ！）
        offsetxy = ci['center'] #円のセンターの座標を取得(x, y, z)
offx = offsetxy[0] #xとyのオフセットが出る
offy = offsetxy[1]
print("offset = ", offx, offy)
lines = [e for e in dxf.entities if e.dxftype == 'LINE']
# 直線
# 以下のデータは図面毎に手入力必要（メモリー
#maxx = [-36200., 104000.] #UCS座標で最大値（これをしないと範囲外のデータもすべてとるのでメモリーオーバーになりそう）
#maxy = [-46600., 73700.] #UCS座標
needlayer = [str(i) for i in range(100)]
datalist = []
for i in range(len(lines)):
    li = vars(lines[i])
    layer = li['layer']
    
    if layer in needlayer:
        datai = [li['start'][0], li['start'][1], int(layer)]
        dataiE = [li['end'][0], li['end'][1], int(layer)]
        x = datai[0] - offx #UCS座標変換
        y = datai[1] - offy #UCS座標変換
        xE = dataiE[0] - offx #UCS座標変換
        yE = dataiE[1] - offy #UCS座標変換
        datai2 = [x, y, int(layer)] #endのポイントは次のstartと重複の可能性が高いので入れない。
        datai2E = [xE, yE, int(layer)] #end
        datalist.append(datai2)
        datalist.append(datai2E)

#重複するデータの削除
result = []
[result.append(x) for x in datalist if x not in result]
datalist = result

#with open('./'+name+'.csv', 'w', newline="") as f: #newlineで一行空けない指定
with open('./'+'Jiyama_all'+'.csv', 'w', newline="") as f:
    writer = csv.writer(f)
    for li in datalist:
        writer.writerow(li)